In [ ]:
novelty_checker_prompt = ''' You are an expert in evaluating research content for novelty and innovation. I have two sets of text
provided below:

Author-Mentioned Future Work:
<Author-Mentioned Future Work>

LLM-Generated Future Work:
<LLM-Generated Future Work>

Your task is to compare the LLM-generated future work to the author-mentioned future work and assess its novelty relative to the
author-mentioned future work. Follow these steps:

Evaluate Novelty: Identify unique ideas, approaches, or directions in the LLM-generated future work that are not present in
the author-mentioned future work. Analyze how innovative or distinct these additions are in the context of the research field.

Quantify Novelty: Provide a novelty score (0-10) for the LLM-generated future work, where 0 indicates complete overlap with the
author’s future work (no new ideas) and 10 indicates entirely new and distinct ideas. Justify the score with a clear reason explaining
the extent of novel contributions or lack thereof.

Present your response in a JSON object with the keys score (integer from 0-10) and reason (a concise explanation of the score).
Ensure your response is concise, precise, and grounded in the provided texts. If the provided texts are incomplete or unclear,
request clarification or specific excerpts to proceed.

Example output format: { "score": 5, "reason": "The LLM-generated future work introduces a new approach to cross-lingual embeddings,
which is not mentioned in the author’s future work, but it also repeats the idea of low-resource language support, reducing its
overall novelty." }

\n\n
'''

In [ ]:
import os
from openai import OpenAI
import time

os.environ['OPENAI_API_KEY'] = ''
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)
generated_summary = []
# abstracts = []

for i in range(len(df_rest)): # len(df_rest)
    #prompt = "Objective: Generate summary for the following passage from a scientific paper. Consider the Abstract, Introduction, and other sections of a scientific paper. Each summary should be 200 to 300 words. \n\n" + "Abstract:\n" + df_cleaned['Abstract'][i] + "\n" + "Introduction:\n" + df_cleaned['Introduction'][i] + "Others:\n" + df_cleaned['col_1'][i] + "\n" +  df_cleaned['col_2'][i] + "\n" + df_cleaned['col_3'][i] + "\n" +  df_cleaned['col_4'][i] + "\n" +  df_cleaned['col_5'][i] + "\n" +  df_cleaned['col_6'][i] + "\n" + df_cleaned['col_7'][i] + "\n"
    author_str = df_rest.at[i, 'LLM_extracted_future_work']
    generated_str = df_rest.at[i, 'RAG_generated_fw_from_paper']

    # author_str = "\n".join(author_list)
    # generated_str = "\n".join(generated_list)

    prompt = (
        novelty_checker_prompt
        + "Author-Mentioned Future Work:\n"
        + author_str
        + "\n\nLLM-Generated Future Work:\n"
        + generated_str
    )
    # prompt =  novelty_checker_prompt + "Author-Mentioned Future Work: " + df_neurips['LLM_extracted_future_work'][i] + "LLM-Generated Future Work: " + df_neurips['LLM_generated_future_work_paper_only'][i]


    summary_text = ""  # Initialize an empty string to collect the limitation text

    stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        stream=True,
        temperature=0.1 # Adjust the temperature as needed, max_tokens=150
    )

    for chunk in stream:
        summary_chunk = chunk.choices[0].delta.content or ""
        # print(limitation_chunk, end="")
        summary_text += summary_chunk  # Append each chunk to the limitation_text

    # print("\n")  # Print a newline for readability
    summary_chunks = []
    summary_chunks.append(summary_text)
    generated_summary.append(summary_chunks)  # Append the collected limitation text to the list


In [ ]:
import pandas as pd
import json
import re

# Flatten the list of lists
flat = [item[0] for item in generated_summary]

# Parse each JSON snippet
records = []
for snippet in flat:
    match = re.search(r"\{.*\}", snippet, flags=re.S)
    if match:
        data = json.loads(match.group(0))
        records.append(data)

# Create DataFrame
df_gpt_judge = pd.DataFrame(records)
